In [1]:
from random import randint
import random
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import pickle as pkl
import math
random.seed(42)

In [2]:
def mutate_simple(dna):
    dna_list = list(dna)
    mutation_site = random.randint(0, len(dna_list) - 1)
    dna_list[mutation_site] = random.choice(list('ATCG'))
    return ''.join(dna_list)

In [3]:
def draw(discrete_probdist):
    """
    Draw random value from discrete probability distribution
    represented as a dict: P(x=value) = discrete_probdist[value].
    """    
    limit = 0
    r = random.random()
    for value in discrete_probdist:
        limit += discrete_probdist[value]
        if r < limit:
            return value

In [4]:
## Takes into account the frecuency of transitions and transversions
def create_markov_chain():
    markov_chain = {}
    for from_base in 'ATGC':
        # Generate random transition probabilities by dividing
        # [0,1] into four intervals of random length
        slice_points = sorted([0] + [random.random()for i in range(3)] + [1])
        transition_probabilities = [slice_points[i+1] - slice_points[i] for i in range(4)]
        transition_probabilities_sorted = sorted(transition_probabilities)
        bases_string = 'ATGC'
        if(from_base == 'A'):
            bases_string = bases_string.replace('G','')
            bases_string = bases_string.replace('A','')
            bases_string = bases_string + 'A'     
            bases_string = bases_string + 'G'            
        if(from_base == 'G'):
            bases_string = bases_string.replace('A','')
            bases_string = bases_string.replace('G','')
            bases_string = bases_string + 'G'            
            bases_string = bases_string + 'A'            
        if(from_base == 'C'):
            bases_string = bases_string.replace('T','')
            bases_string = bases_string.replace('C','')
            bases_string = bases_string + 'C'
            bases_string = bases_string + 'T'
        if(from_base == 'T'):
            bases_string = bases_string.replace('C','')
            bases_string = bases_string.replace('T','')
            bases_string = bases_string + 'T'   
            bases_string = bases_string + 'C'           
        markov_chain[from_base] = {base: p for base, p in zip(bases_string, transition_probabilities_sorted)}
    return markov_chain

In [5]:
# Pointwise mutation
def mutate_via_markov_chain(dna, markov_chain,mutation_site):       
    dna_list = list(dna)
    from_base = dna[mutation_site]
    if(from_base == 'N'):
        return dna
    to_base = draw(markov_chain[from_base])
    dna_list[mutation_site] = to_base
    return ''.join(dna_list)

In [6]:
def generate_random_sequence(length,gc_percent):
    dna_sequence = ''
    for i in range(0,length):
        step = random.random()
        next_nucleotide = ''
        if(step < gc_percent):
            next_nucleotide =  random.choice(list('CG'))
        else:
            next_nucleotide =  random.choice(list('AT'))
        dna_sequence += next_nucleotide
    return dna_sequence

In [7]:
def get_gc_count(sequence):
    count = 0
    for c in sequence:
        if(c == 'G'):
            count = count + 1
        if(c == 'C'):
            count = count + 1            
    return count    

In [8]:
def insert_str(string, str_to_insert, index):
    return string[:index] + str_to_insert + string[index:]

In [9]:
def mutate_insertion(dna,gc_percent,insertion_length_max,mutation_site):
    return insert_str(dna,generate_random_sequence(randint(1, insertion_length_max),gc_percent),mutation_site)        

In [10]:
def mutate_tandem_insertion(dna,tandem_length_max,tandem_quantity_max,mutation_site):    
    tandem_length = randint(1, tandem_length_max)
    final_position = mutation_site + tandem_length
    insert_string = dna[mutation_site:final_position]
    tandem_length = randint(1, tandem_quantity_max)
    for i in range(1,tandem_length):
        dna = insert_str(dna,insert_string,mutation_site) 
    return dna

In [11]:
def mutate_deletion(dna,deletion_length_max,mutation_site):
    deletion_length =  randint(1, deletion_length_max)
    deletion_position = mutation_site + deletion_length
    return dna[0:mutation_site] + dna[deletion_position:]    

In [12]:
# Likelyhood of indels are between 16% and 25%
def generate_mutated_sequence(dna_sequence,mutation_rate):
    mc = create_markov_chain()
    dna_length = len(dna_sequence)
    nr_mutations = round(dna_length * mutation_rate)  
    for i in range(0,nr_mutations):
        threshold = random.random()
        mutation_site = random.randint(0, len(dna_sequence) - 1)
        if(threshold<0.8):
            dna_sequence = mutate_via_markov_chain(dna_sequence,mc,mutation_site)
        else:
            threshold = random.random()
            ## Deletions are more likely than insertions (Zhang,2003)
            sequenceLength = min(random.randint(1,51),round(0.05*dna_length))
            if(threshold < 0.7):                
                dna_sequence = mutate_deletion(dna_sequence,sequenceLength,mutation_site)
            else:
                ## Tandem mutations are more likely
                threshold = random.random()
                if(threshold<0.75):
                    dna_sequence = mutate_tandem_insertion(dna_sequence,sequenceLength,random.randint(1,5),mutation_site)                    
                else:
                    dna_sequence = mutate_insertion(dna_sequence,get_gc_count(dna_sequence)/float(len(dna_sequence)),sequenceLength,mutation_site)      
    return dna_sequence

In [17]:
def get_df_dict(level):
    df_dict = {}
    df_dict["Ohnologs"] = {}
    df_dict["No-Ohnologs"] = {}
    df_dict["Paralogs"] = {}
    for animal in animalList:        
        current_file_path = filepath + animal + "/" + level + "/"
        df_dict["Ohnologs"][animal] = pd.read_pickle(current_file_path + animal + "-" + level +"-ohnologs.pkl")
        df_dict["No-Ohnologs"][animal] = pd.read_pickle(current_file_path + animal + "-" + level +"-no-ohnologs.pkl")
        df_dict["Paralogs"][animal] = pd.read_pickle(current_file_path + animal + "-" + level +"-paralogues.pkl")        
    return df_dict

In [19]:
df_dict = get_df_dict("Strict")
df_dict["Ohnologs"]["Human"].head()

,Ohnolog-1 Id,Ohnolog-2 Id,Ohnolog-1 Symbol,Ohnolog-2 Symbol,Synteny Outgroup Support,Combine q-value(self) from all vetebrates,Combine q-value(outgroup) from all vetebrates,q-value from self comparison,Combined q-value for all outgroups,q-value for Amphioxus,...,q-value for Worm,Duplication node form Ensembl,Ohnolog-1/Sequence-Lenght,Ohnolog-1/Sequence,Ohnolog-2/Sequence-Lenght,Ohnolog-2/Sequence,Ohnolog-1/Transcript-ID,Ohnolog-2/Transcript-ID,GC_Percent_1,GC_Percent_2
0,ENSG00000095464,ENSG00000132915,PDE6C,PDE6A,4.0,8.228000e-06,0.000076,7.960000e-12,1.090000e-09,2.630000e-08,...,0.9510,Vertebrata,3307,CTTTGGAAGTCCTATGAGGGACCATTTACGGTTTCCTCAGTAATTT...,5706,AGTATGTTTTGCAGACAAGACCCAGAGAAGTCCAGACTGGACTTGT...,ENST00000371447,ENST00000508173,0.429090,0.469856
1,ENSG00000077684,ENSG00000102221,PHF17,PHF16,5.0,1.900000e-03,0.000010,1.070000e-02,4.790000e-07,3.370000e-01,...,0.0789,Vertebrata,5772,CGTTTTGGCAAGGGATTAAAGTGCTCCCCCCTGTGGCAGCAGTGAC...,4934,ATACAATAGTGCTCCGCGCCGCCTCAGCCGCCGCCGCCGCCCAACC...,ENST00000226319,ENST00000614628,0.441268,0.450953
2,ENSG00000109158,ENSG00000145863,GABRA4,GABRA6,5.0,1.923000e-09,0.000013,1.340000e-13,2.370000e-07,4.000000e-04,...,NaN,Vertebrata,11973,AGTCAACCTCTGGAAGTAAGTCAACTCCATTCTGAAAAAGAAGAGT...,2393,ACATAATCTAAGACCACAAACCACCTTGTTCCACGTGAGAAGGAAA...,ENST00000264318,ENST00000523217,0.357972,0.402006
3,ENSG00000130758,ENSG00000173327,MAP3K10,MAP3K11,5.0,8.785000e-07,0.004300,3.700000e-06,3.300000e-03,5.444000e-01,...,0.0853,Vertebrata,3436,CGCGCGGCCAGGCCCTCTTAGCCCTCTGCCGTTTGGGGGGCACGGG...,5605,GAAGAAGGGAGCGGGGTCGGAGCCGTCGGGGCCAAAGGAGACGGGG...,ENST00000253055,ENST00000527304,0.684226,0.591079
4,ENSG00000166562,ENSG00000140612,SEC11C,SEC11A,3.0,2.697000e-06,0.009500,9.420000e-07,1.140000e-02,1.457000e-01,...,0.1060,Vertebrata,2054,CGGTGGGCGGGGGCCGGCAGGTGCTCCGCAGCCGTCTGTGCCACCC...,3626,AGCGATTCTGCTGCCACAACCTCCTGAGTAGCTGGGATTACAGGCG...,ENST00000509791,ENST00000558924,0.402629,0.493657


In [34]:
def get_ohnologs_mutations_one(df_ohnologs_to_mutate,mutation_rate):    
    new_sequence_1 = []
    new_sequence_1_len = []
    new_sequence_1_gc = []
    new_sequence_2 = []
    new_sequence_2_len = []
    new_sequence_2_gc = []
    mutated_sequence_nr = []
    
    for index, row in df_ohnologs_to_mutate.iterrows():        
        threshold = random.random()
        if(threshold<0.5):              
            mutated_sequence = generate_mutated_sequence(row["Sequence-1"],mutation_rate)
            new_sequence_1.append(mutated_sequence)
            new_sequence_1_len.append(len(mutated_sequence))
            new_sequence_1_gc.append(get_gc_count(mutated_sequence)/float(len(mutated_sequence)))
            
            new_sequence_2.append(row['Sequence-2'])
            new_sequence_2_len.append(row['Sequence-2 Length'])
            new_sequence_2_gc.append(row['Sequence-2 GC'])   
            mutated_sequence_nr.append(1)
        else:
            new_sequence_1.append(row['Sequence-1'])
            new_sequence_1_len.append(row['Sequence-1 Length'])
            new_sequence_1_gc.append(row['Sequence-1 GC'])
            
            mutated_sequence = generate_mutated_sequence(row["Sequence-2"],mutation_rate)
            new_sequence_2.append(mutated_sequence)
            new_sequence_2_len.append(len(mutated_sequence))
            new_sequence_2_gc.append(get_gc_count(mutated_sequence)/float(len(mutated_sequence)))
            mutated_sequence_nr.append(2)
            
    df_ohnologs_to_mutate["Sequence-1-Mutated"] = new_sequence_1
    df_ohnologs_to_mutate["Sequence-1 Length-Mutated"] = new_sequence_1_len
    df_ohnologs_to_mutate["Sequence-1 GC-Mutated"] = new_sequence_1_gc

    df_ohnologs_to_mutate["Sequence-2-Mutated"] = new_sequence_2
    df_ohnologs_to_mutate["Sequence-2 Length-Mutated"] = new_sequence_2_len
    df_ohnologs_to_mutate["Sequence-2 GC-Mutated"] = new_sequence_2_gc

    df_ohnologs_to_mutate["Mutated_Sequence_Nr"] = mutated_sequence_nr    
    return df_ohnologs_to_mutate

In [35]:
def get_ohnologs_mutations_two(df_ohnologs_to_mutate,mutation_rate):        
    new_sequence_1 = []
    new_sequence_1_len = []
    new_sequence_1_gc = []
    new_sequence_2 = []
    new_sequence_2_len = []
    new_sequence_2_gc = []
    mutated_sequence_nr = []
    
    for index, row in df_ohnologs_to_mutate.iterrows():                        
        mutated_sequence = generate_mutated_sequence(row["Sequence-1"],mutation_rate)
        new_sequence_1.append(mutated_sequence)
        new_sequence_1_len.append(len(mutated_sequence))
        new_sequence_1_gc.append(get_gc_count(mutated_sequence)/float(len(mutated_sequence)))

        mutated_sequence = generate_mutated_sequence(row["Sequence-2"],mutation_rate)
        new_sequence_2.append(mutated_sequence)
        new_sequence_2_len.append(len(mutated_sequence))
        new_sequence_2_gc.append(get_gc_count(mutated_sequence)/float(len(mutated_sequence)))
        
        mutated_sequence_nr.append(0)
                        
    df_ohnologs_to_mutate["Sequence-1-Mutated"] = new_sequence_1
    df_ohnologs_to_mutate["Sequence-1 Length-Mutated"] = new_sequence_1_len
    df_ohnologs_to_mutate["Sequence-1 GC-Mutated"] = new_sequence_1_gc

    df_ohnologs_to_mutate["Sequence-2-Mutated"] = new_sequence_2
    df_ohnologs_to_mutate["Sequence-2 Length-Mutated"] = new_sequence_2_len
    df_ohnologs_to_mutate["Sequence-2 GC-Mutated"] = new_sequence_2_gc

    df_ohnologs_to_mutate["Mutated_Sequence_Nr"] = mutated_sequence_nr        
    
    return df_ohnologs_to_mutate

In [36]:
def get_ohnologs_mutations(df_animal,level):
    ## We mutatate 10% of the sample
    df_ohnologs_to_mutate = df_animal.sample(round(len(df_animal)*0.1),random_state=42)    
    ## We only mutate one sequence    
    if(level == "Low" or level == "Medium"):        
        df_ohnologs_to_mutate = get_ohnologs_mutations_one(df_ohnologs_to_mutate,mutation_rate[level])                    
    else:
    # We mutate both the sequences
        df_ohnologs_to_mutate = get_ohnologs_mutations_two(df_ohnologs_to_mutate,mutation_rate[level])        
    return df_ohnologs_to_mutate

In [37]:
def standarize_df(df):
    df_temp = pd.DataFrame(columns=['Sequence-1 Id','Sequence-2 Id','Sequence-1','Sequence-2','Sequence-1 Length','Sequence-2 Length','Sequence-1 GC','Sequence-2 GC','Is_Ohnolog'])
    df_temp['Sequence-1 Id'] = df["Ohnolog-1 Id"]
    df_temp['Sequence-2 Id'] = df["Ohnolog-2 Id"]
    df_temp['Sequence-1-Transcript Id'] = df["Ohnolog-1/Transcript-ID"]
    df_temp['Sequence-2-Transcript Id'] = df["Ohnolog-2/Transcript-ID"]
    df_temp['Sequence-1'] = df["Ohnolog-1/Sequence"]
    df_temp['Sequence-2'] = df["Ohnolog-2/Sequence"]
    df_temp['Sequence-1 Length'] = df["Ohnolog-1/Sequence-Lenght"]
    df_temp['Sequence-2 Length'] = df["Ohnolog-2/Sequence-Lenght"]
    df_temp['Sequence-1 GC'] = df["GC_Percent_1"]
    df_temp['Sequence-2 GC'] = df["GC_Percent_2"]
    df_temp['Is_Ohnolog'] = 1
    return df_temp

In [16]:
filepath = "Animals/"
animalList = ["Human","Pig","Chicken","Rat","Mouse","Dog"]
levelList = ["Strict","Relaxed","Intermediate"]

In [39]:
mutation_levels = ["VeryLow","Low","Medium","High"]
# Mutation rate per year for human. Most researched. Similar to other mammals
mutation_rate = {}
mutation_rate["VeryLow"] = 0.01
mutation_rate["Low"] = 0.025
mutation_rate["Medium"] = 0.05
mutation_rate["High"] = 0.1

In [40]:
for level in levelList:    
    df_animals_dict = get_df_dict(level)
    for animal in animalList:
        for mutation_level in mutation_levels:
            df_mutated = get_ohnologs_mutations(standarize_df(df_animals_dict["Ohnologs"][animal]),mutation_level)
            current_file_path = filepath + animal + "/" + level + "/" + "mutations/"
            df_mutated.to_pickle(current_file_path + animal + "-" + level + "-mutated-" + mutation_level + "-ohnologs.pkl")                        